# ДОМАШНЕЕ ЗАДАНИЕ 4. Рекомендательные системы и Spark MLlib

**Дисциплина:** Методы обработки больших данных  
**Студент:** Попов Я.Ю. 
**Группа:** ИУ6-31М

## Решение для полного набора данных

### Задание 1. Анализ датасета

1. Выведите данные, сопоставляющие жанры и количество фильмов
2. Выведите первые 10 фильмов с наибольшим количеством рейтингов для каждого жанра в соответствии с вариантом
3. Выведите первые 10 фильмов с наименьшим количеством рейтингов (но больше 10) для каждого жанра в соответствии с вариантом
4. Выведите первые 10 фильмов с наибольшим средним рейтингом при количестве рейтингов больше 10 для каждого жанра в соответствии с вариантом
5. Выведите первые 10 фильмов с наименьшим средним рейтингом при количестве рейтингов больше 10 для каждого жанра в соответствии с вариантом


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, count, avg, row_number
from pyspark.sql.window import Window

# Инициализация SparkSession
spark = SparkSession.builder \
    .appName("MovieRecommendationAnalysis") \
    .getOrCreate()

# Загрузка данных
movies_df = spark.read.csv("movies.csv", header=True, inferSchema=True)
ratings_df = spark.read.csv("ratings.csv", header=True, inferSchema=True)

# Список требуемых жанров
genres = [
    "Animation", "Romance", "Documentary", "Drama", 
    "Comedy", "Musical", "Thriller", "Sci-Fi", "Adventure"
]

/home/ubuntu/ML/anaconda3/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


## Задача 1: Количество фильмов по жанрам

In [4]:
# Разделение на жанры по "|"
movies_exploded = movies_df.withColumn("genre", explode(split(col("genres"), "\\|")))
genre_counts = (
    movies_exploded
    .filter(col("genre").isin(genres))
    .groupBy("genre")
    .count()
    .orderBy(col("count").desc())
)

print("Задача 1: Количество фильмов по жанрам")
genre_counts.show()

Задача 1: Количество фильмов по жанрам
+-----------+-----+
|      genre|count|
+-----------+-----+
|      Drama|33681|
|     Comedy|22829|
|   Thriller|11675|
|    Romance|10172|
|Documentary| 9283|
|  Adventure| 5349|
|     Sci-Fi| 4850|
|  Animation| 4579|
|    Musical| 1059|
+-----------+-----+



## Задача 2: Топ-10 фильмов с наибольшим количеством рейтингов для каждого жанра

In [5]:
# Агрегация рейтингов
movie_ratings = (
    ratings_df
    .groupBy("movieId")
    .agg(
        count("rating").alias("rating_count"),
        avg("rating").alias("avg_rating")
    )
)

# Объединение с информацией о фильмах
movies_enriched = (
    movies_df
    .join(movie_ratings, "movieId", "inner")
    .withColumn("genre", explode(split(col("genres"), "\\|")))
    .filter(col("genre").isin(genres))
)


window_desc = Window.partitionBy("genre").orderBy(col("rating_count").desc())
top_by_count = (
    movies_enriched
    .withColumn("rank", row_number().over(window_desc))
    .filter(col("rank") <= 10)
    .select("genre", "title", "rating_count", "rank")
    .orderBy("genre", "rank")
)

print("\nЗадача 2: Топ-10 по количеству рейтингов")
top_by_count.show(n=100, truncate=False)


Задача 2: Топ-10 по количеству рейтингов
+-----------+------------------------------------------------------------------------------+------------+----+
|genre      |title                                                                         |rating_count|rank|
+-----------+------------------------------------------------------------------------------+------------+----+
|Adventure  |Star Wars: Episode IV - A New Hope (1977)                                     |97202       |1   |
|Adventure  |Jurassic Park (1993)                                                          |83026       |2   |
|Adventure  |Star Wars: Episode V - The Empire Strikes Back (1980)                         |80200       |3   |
|Adventure  |Lord of the Rings: The Fellowship of the Ring, The (2001)                     |79940       |4   |
|Adventure  |Toy Story (1995)                                                              |76813       |5   |
|Adventure  |Star Wars: Episode VI - Return of the Jedi (1983)        

## Задача 3: Топ-10 фильмов с наименьшим количеством рейтингов (>10) для каждого жанра

In [6]:

window_asc = Window.partitionBy("genre").orderBy(col("rating_count").asc())
filtered_movies = movies_enriched.filter(col("rating_count") > 10)
bottom_by_count = (
    filtered_movies
    .withColumn("rank", row_number().over(window_asc))
    .filter(col("rank") <= 10)
    .select("genre", "title", "rating_count", "rank")
    .orderBy("genre", "rank")
)
print("\nЗадача 3: Наименьшее количество рейтингов (>10)")
bottom_by_count.show(truncate=False)


Задача 3: Наименьшее количество рейтингов (>10)
+---------+------------------------------------------------------+------------+----+
|genre    |title                                                 |rating_count|rank|
+---------+------------------------------------------------------+------------+----+
|Adventure|Edge of the World (2021)                              |11          |1   |
|Adventure|Righteous Ties (2006)                                 |11          |2   |
|Adventure|A Tourist's Guide to Love (2023)                      |11          |3   |
|Adventure|Frontier (2018)                                       |11          |4   |
|Adventure|The Life and Death of a Porno Gang (2009)             |11          |5   |
|Adventure|Blackie the Pirate (1971)                             |11          |6   |
|Adventure|To Kill a King (2003)                                 |11          |7   |
|Adventure|K2: Siren of the Himalayas (2012)                     |11          |8   |
|Adventure|Journ

## Задача 4: Топ-10 фильмов с наибольшим средним рейтингом (>10 рейтингов)

In [7]:
window_avg_desc = Window.partitionBy("genre").orderBy(
    col("avg_rating").desc(),
    col("rating_count").desc()
)
top_by_avg = (
    filtered_movies
    .withColumn("rank", row_number().over(window_avg_desc))
    .filter(col("rank") <= 10)
    .select("genre", "title", "avg_rating", "rating_count", "rank")
    .orderBy("genre", "rank")
)
print("\nЗадача 4: Наивысший средний рейтинг (>10)")
top_by_avg.show(truncate=False)


Задача 4: Наивысший средний рейтинг (>10)
+---------+----------------------------------------------------+------------------+------------+----+
|genre    |title                                               |avg_rating        |rating_count|rank|
+---------+----------------------------------------------------+------------------+------------+----+
|Adventure|Over the Garden Wall (2013)                         |4.256993006993007 |1430        |1   |
|Adventure|Spider-Man: Across the Spider-Verse (2023)          |4.252840909090909 |528         |2   |
|Adventure|Seven Samurai (Shichinin no samurai) (1954)         |4.2508177570093455|17120       |3   |
|Adventure|Adventure Time: Elements (2017)                     |4.25              |12          |4   |
|Adventure|Spirited Away (Sen to Chihiro no kamikakushi) (2001)|4.226035335689046 |35375       |5   |
|Adventure|Spider-Man: Into the Spider-Verse (2018)            |4.192053284336242 |10885       |6   |
|Adventure|North by Northwest (1959)   

## Задача 5: Топ-10 фильмов с наименьшим средним рейтингом (>10 рейтингов)

In [8]:
window_avg_asc = Window.partitionBy("genre").orderBy(
    col("avg_rating").asc(),
    col("rating_count").desc()
)
bottom_by_avg = (
    filtered_movies
    .withColumn("rank", row_number().over(window_avg_asc))
    .filter(col("rank") <= 10)
    .select("genre", "title", "avg_rating", "rating_count", "rank")
    .orderBy("genre", "rank")
)
print("\nЗадача 5: Наименьший средний рейтинг (>10)")
bottom_by_avg.show(n=100, truncate=False)


Задача 5: Наименьший средний рейтинг (>10)
+-----------+------------------------------------------------------------------------+------------------+------------+----+
|genre      |title                                                                   |avg_rating        |rating_count|rank|
+-----------+------------------------------------------------------------------------+------------------+------------+----+
|Adventure  |Kidnapping, Caucasian Style (2014)                                      |0.9117647058823529|17          |1   |
|Adventure  |Titanic 2 (2010)                                                        |1.0454545454545454|22          |2   |
|Adventure  |Bigfoot (2012)                                                          |1.0588235294117647|17          |3   |
|Adventure  |Beethoven's Treasure Tail (2014)                                        |1.1666666666666667|15          |4   |
|Adventure  |Barney's Great Adventure (1998)                                         |1.

## Остановка SparkSession

In [9]:
spark.stop()

## Задание 2. Коллаборативная фильтрация

Вариант 1. По схожести пользователей

Вариант 2. По схожести объектов

1. Разделите данные с рейтингами на обучающее (train_init - 0.8) и тестовое подмножества (test - 0.2), определите среднее значение рейтинга в обучающем подмножестве и вычислите rmse для тестового подмножества, если для всех значений из test предсказывается среднее значение рейтинга
2. Реализуйте коллаборативную фильтрацию в соответствии с вариантом. Для определения схожести используйте train_init, для расчета rmse - test
3. Определите rmse для тестового подмножества


In [1]:
surname = "Попов" 

alp = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
w = [1, 42, 21, 21, 34,  6, 44, 26, 18, 44, 38, 26, 14, 43,  4, 49, 45,
        7, 42, 29,  4,  9, 36, 34, 31, 29,  5, 30,  4, 19, 28, 25, 33]

d = dict(zip(alp, w))
variant =  sum([d[el] for el in surname.lower()]) % 40 + 1

print("Задача № 2: ", variant % 2 + 1 )

Задача № 2:  1


## Инициализация SparkSession

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, avg, sqrt, broadcast
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
import time

# Инициализация SparkSession для работы с полным набором данных
spark = SparkSession.builder \
    .appName("MovieLens_FullDataset_Analysis") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.default.parallelism", "200") \
    .config("spark.driver.maxResultSize", "4g") \
    .config("spark.memory.fraction", "0.8") \
    .getOrCreate()

ratings_df = spark.read.csv("ratings.csv", header=True, inferSchema=True)

# Кэширование данных для ускорения обработки
ratings_df = ratings_df.cache()

/home/ubuntu/ML/anaconda3/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


## 1: Разделение данных и вычисление RMSE для среднего значения

In [3]:
# Разделение данных
train_init, test = ratings_df.randomSplit([0.8, 0.2], seed=42)

# Вычисление среднего рейтинга в обучающей выборке
start_time = time.time()
avg_rating = train_init.agg(avg("rating").alias("avg_rating")).collect()[0]["avg_rating"]
print(f"Средний рейтинг в обучающей выборке: {avg_rating:.4f}")
print(f"Вычислено за {time.time() - start_time:.2f} секунд")

# Создание предсказаний для тестового набора
test_with_pred = test.withColumn("prediction", lit(avg_rating))

# Расчет RMSE вручную
start_time = time.time()
rmse_base = test_with_pred.withColumn("squared_error", (col("rating") - col("prediction")) ** 2)
rmse_base = rmse_base.agg(sqrt(avg("squared_error")).alias("rmse")).collect()[0]["rmse"]
print(f"RMSE базовой модели (предсказание средним): {rmse_base:.4f}")
print(f"RMSE рассчитано за {time.time() - start_time:.2f} секунд")

Средний рейтинг в обучающей выборке: 3.5426
Вычислено за 34.94 секунд
RMSE базовой модели (предсказание средним): 1.0641
RMSE рассчитано за 4.14 секунд


## 2: Реализация коллаборативной фильтрации по схожести пользователей

In [4]:
# Подготовка данных для ALS
als_train = train_init.select(
    col("userId").cast("integer").alias("user"),
    col("movieId").cast("integer").alias("item"),
    col("rating").cast("float")
).cache()

als_test = test.select(
    col("userId").cast("integer").alias("user"),
    col("movieId").cast("integer").alias("item"),
    col("rating").cast("float")
).cache()

# Настройка параметров ALS 
als = ALS(
    maxIter=15,
    regParam=0.1,
    rank=10,
    userCol="user",
    itemCol="item",
    ratingCol="rating",
    coldStartStrategy="drop",
    nonnegative=True,
    seed=42
)

# Обучение ALS модели
start_time = time.time()
model = als.fit(als_train)
training_time = time.time() - start_time
print(f"Модель обучена за {training_time:.2f} секунд")

# Предсказание рейтинов для тестового набора
start_time = time.time()
predictions = model.transform(als_test)
prediction_time = time.time() - start_time
print(f"Предсказания сгенерированы за {prediction_time:.2f} секунд")

Модель обучена за 125.95 секунд
Предсказания сгенерированы за 0.25 секунд


## 3: Определение RMSE для тестового подмножества

In [5]:
# Расчет RMSE
start_time = time.time()
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)

rmse_als = evaluator.evaluate(predictions)
print(f"RMSE ALS модели: {rmse_als:.4f}")

# Альтернативный расчет RMSE вручную для валидации
rmse_manual = predictions.withColumn("squared_error", (col("rating") - col("prediction")) ** 2)
rmse_manual = rmse_manual.agg(sqrt(avg("squared_error")).alias("rmse")).collect()[0]["rmse"]
print(f"RMSE (ручной расчет): {rmse_manual:.4f}")
print(f"Время оценки: {time.time() - start_time:.2f} секунд")

print("\nСРАВНЕНИЕ МОДЕЛЕЙ")
print(f"Базовая модель (глобальное среднее):")
print(f"  RMSE = {rmse_base:.4f}")
print(f"ALS модель (коллаборативная фильтрация):")
print(f"  RMSE = {rmse_als:.4f}")
print(f"  Улучшение: {rmse_base - rmse_als:.4f} ({(rmse_base - rmse_als)/rmse_base*100:.2f}%)")

RMSE ALS модели: 0.8142
RMSE (ручной расчет): 0.8142
Время оценки: 65.18 секунд

СРАВНЕНИЕ МОДЕЛЕЙ
Базовая модель (глобальное среднее):
  RMSE = 1.0641
ALS модель (коллаборативная фильтрация):
  RMSE = 0.8142
  Улучшение: 0.2499 (23.49%)


In [6]:
ratings_df.unpersist()
als_train.unpersist()
als_test.unpersist()

spark.stop()